In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sys import stdout

from importlib import reload

In [ ]:
import ezmc
reload(ezmc)
reload(ezmc.base)
reload(ezmc.samplers)
reload(ezmc.utils)
import ezmc


In [ ]:
def f(x):
    a, b = x
    return stats.norm.logpdf(a, 0, 1) + stats.norm.logpdf(b, 0, 1)

def f(x):
    v1, v2, r = 1, 1, .9
    dist = stats.multivariate_normal(mean=[0, 0], cov=[[v1*v1, v1*r], [v2*r, v2*v2]])
    return dist.logpdf(x)

def sample_true(n):
    v1, v2, r = 1, 1, .9
    dist = stats.multivariate_normal(mean=[0, 0], cov=[[v1*v1, v1*r], [v2*r, v2*v2]])
    return dist.rvs(n)

def init_func():
    return np.random.normal(0, 5, 2)

In [ ]:
true_posterior = sample_true(1000).T
plt.scatter(true_posterior[0], true_posterior[1])

# DEMC

In [ ]:
# ezmc.DifferentialEvolutionSampler?

In [ ]:
m = ezmc.DifferentialEvolutionSampler(func=f, 
                                      par_names=['a', 'b'],
                                     init_bounds=[[-10, 10],
                                                 [-10, 10]])

In [ ]:
m.sample_chains(nchains=10, n=20000, verbose=1)

In [ ]:
chains = m.get_chains()
ezmc.viz.traceplot(chains)

In [ ]:
results = m.get_results(burn_in=500, thin=20)
ezmc.viz.traceplot(results, ['a', 'b'])

In [ ]:
plt.scatter(results['a'], results['b'])

In [ ]:
print(np.mean(true_posterior, 1), np.std(true_posterior, 1), np.corrcoef(true_posterior)[0,1])

In [ ]:
a = results[['a', 'b']].values.T
print(np.mean(a, 1), np.std(a, 1), np.corrcoef(a)[0,1])

In [ ]:
def kde_countour(x, y, colors='k'):
    deltaX = (max(x) - min(x))/10
    deltaY = (max(y) - min(y))/10
    xmin = min(x) - deltaX
    xmax = max(x) + deltaX
    ymin = min(y) - deltaY
    ymax = max(y) + deltaY
    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x, y])
    kernel = stats.gaussian_kde(values)
    f = np.reshape(kernel(positions).T, xx.shape)
    cset = plt.contour(xx, yy, f, colors=colors)
    
kde_countour(results['a'], results['b'], colors='red')
kde_countour(p[0], p[1], colors='blue')

In [ ]:
np.sin(p[0, 0], p[1, 0])

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(results['a']);

In [ ]:
ezmc.viz.traceplot(results, ['ll'])

In [ ]:
raise Exception

# Multiple Chains

In [ ]:
m = ezmc.MetropolisSampler(func=f, par_names=['a', 'b'], proposal_sd=[.25, .25], init_func=init_func)
m.sample_chains(4, n=3000, verbose=0)

In [ ]:
chains = m.get_chains()

In [ ]:
ezmc.viz.traceplot(chains, ['a', 'b'])

In [ ]:
results = m.get_results(burn_in=500, thin=2)

In [ ]:
traceplot(results)

In [ ]:
results.mean()

In [ ]:
results.std()

# One Chain

In [ ]:
m = ezmc.MetropolisSampler(func=f, par_names=['a', 'b'], proposal_sd=[.25, .25], init_func=init_func)
m.add_chains(1)

In [ ]:
m.sample_chain(chain_ix=0, n=100)

In [ ]:
chains = m.chains[0].get_results(burn_in=0, thin=1)
chains[['a', 'b']].plot()

In [ ]:
results = m.chains[0].get_results(burn_in=2000, thin=10)
results.head()
results[['a', 'b']].hist(bins=10)

In [ ]:
print(results.mean())
print(results.std())

In [ ]:
plt.scatter(results['a'], results['ll'])

In [ ]:
plt.scatter(results['b'], results['ll'])

In [ ]:
stats.pearsonr(results['a'], results['b'])

In [ ]:
plt.scatter(results['a'], results['b'])

In [ ]:
chains['ll'].plot()

In [ ]:
m = ezmc.MetropolisSampler

In [ ]:
chain.chain